In [1]:
# The below gherkin feature file text is here as a reference for the implementation
# You would typically have these features nailed down with your stakeholders before
# creating code if you follow DDD best practices

feature_events = """
Feature: Todo Domain Events

    Scenario: TodoAdded
        Given a TodoAdded event
        Then the event data has a id
        Then the event data has a description
        And the event data has a status

    Scenario: TodoStatusChanged
        Given a TodoStatusChanged event
        Then the event data has a id
        And the event data has a status

    Scenario: TodoRemoved
        Given a TodoRemoved event
        Then the event data has a id
"""

feature_commands = """
Feature: Todo Domain Commands

    Scenario: AddTodo
        Given a AddTodo command
        Then the command data has a description
        And the command data has a status

    Scenario: ChangeTodoStatus
        Given a ChangeTodoStatus command
        Then the command data has a id
        And the command data has a status

    Scenario: RemoveTodo
        Given a RemoveKey command
        Then the command data has a id
"""

feature_state = """
Feature: Todo State

Setting and removing a todo updates the 

    Scenario: A Todo is added
        Given a todo
        When a todo is added
        Then the todo has a id
        And the todo state has a description
        And the todo state has a status
        And the todo state has a added_ts

    Scenario: A todo status is changed
        Given a todo
        When the todo status is changed
        Then the todo state status is changed to the new status
        And the todo state has a changed_ts

    Scenario: A todo is removed
        Given a todo
        When a todo is removed
        Then the todo state has a removed_ts
"""

In [2]:
# Imports for building the implementation models
from datetime import datetime
from functools import lru_cache as memoize
from pprint import pprint
from pyrsistent import pmap, pset, pvector, PClass, thaw

from dvent.aggregate import Aggregate
from dvent.command import Command
from dvent.command_handler import CommandHandler
from dvent.event import Event
from dvent.event_store import InMemoryEventStore
from dvent.repository import Repository

In [3]:
# Define the Events and Commands
class TodoEvents(PClass):
    ADDED = 'TodoAdded'
    STATUS_CHANGED = 'TodoStatusChanged'
    REMOVED = 'TodoRemoved'

    @classmethod
    def added(cls, description, status):
        return Event.generate(
            cls.ADDED, data=pmap({
                'description': description,
                'status': status,
            })
        )

    @classmethod
    def status_changed(cls, id, status):
        return Event.generate(
            cls.STATUS_CHANGED, data=pmap({
                'id': id,
                'status': status,
            })
        )

    @classmethod
    def removed(cls, id):
        return Event.generate(
            cls.REMOVED, data=pmap({
                'id': id,
            })
        )

class TodoCommands(PClass):
    ADD = 'AddTodo'
    CHANGE_STATUS = 'ChangeTodoStatus'
    REMOVE = 'RemoveTodo'

    @classmethod
    def add(cls, description, status):
        return Command.generate(
            cls.ADD, data=pmap({
                'description': description,
                'status': status,
            })
        )

    @classmethod
    def change_status(cls, id, status):
        return Command.generate(
            cls.CHANGE_STATUS, data=pmap({
                'id': id,
                'status': status,
            })
        )

    @classmethod
    def remove(cls, id):
        return Command.generate(
            cls.REMOVE, data=pmap({
                'id': id,
            })
        )

In [4]:
# Define the Aggregate implementation
class Todo(Aggregate):
    """
    Todo items
    """

    @classmethod
    @memoize(maxsize=1)  # Once this is generated we don't need to do so again
    def get_apply_map(cls):
        """
        Map event types to handler functions
        """
        return pmap({
            TodoEvents.ADDED: cls.apply_added,
            TodoEvents.STATUS_CHANGED: cls.apply_status_changed,
            TodoEvents.REMOVED: cls.apply_removed,
        })
    
    @staticmethod
    def _validate_status(status):
        assert status in ("Queued", "Started", "Finished")
    
    @classmethod
    def add(cls, description, status):
        cls._validate_status(status)

        _agg = cls.generate()
        return _agg.apply_event(
            TodoEvents.added(
                description=description,
                status=status,
            )
        )

    def change_status(self, new_status):
        """
        Status must be one of "Queued", "Started", "Finished"
        """
        self._validate_status(new_status)

        # If no change then no-op
        if self.state['status'] == new_status:
            return self

        return self.apply_event(
            TodoEvents.status_changed(
                id=self.id,
                status=new_status
            )
        )
    
    def remove(self):
        if self.state.get('removed_ts'):
            return self

        return self.apply_event(
            TodoEvents.removed(
                id=self.id,
            )
        )

    @staticmethod
    def apply_added(todo, event):
        return todo\
            .set_state('description', event.data['description'])\
            .set_state('status', event.data['status'])\
            .set_state('added_ts', event.timestamp)

    @staticmethod
    def apply_status_changed(todo, event):
        return todo\
            .set_state('status', event.data['status'])\
            .set_state('changed_ts', event.timestamp)

    @staticmethod
    def apply_removed(todo, event):
        return todo\
            .set_state('removed_ts', event.timestamp)


In [5]:
# Define the command handler
class TodoCommandHandler(CommandHandler):
    """
    Handles commands within the provided context
    """

    @classmethod
    @memoize(maxsize=1)
    def get_handle_map(cls):
        return pmap({
            TodoCommands.ADD: cls.add,
            TodoCommands.CHANGE_STATUS: cls.change_status,
            TodoCommands.REMOVE: cls.remove,
        })

    # Command handler functions
    @staticmethod
    def add(context, command):
        repository = context.repository
        return repository.save_aggregate(
            Todo.add(
                description=command.data['description'],
                status=command.data['status'],
            )
        )

    @staticmethod
    def change_status(context, command):
        repository = context.repository
        todo = repository.get_aggregate(Todo, command.data['id'])
        return repository.save_aggregate(
            todo.change_status(new_status=command.data['status'])
        )

    @staticmethod
    def remove(context, command):
        repository = context.repository
        todo = repository.get_aggregate(Todo, command.data['id'])
        return repository.save_aggregate(todo.remove())


In [6]:
event_store = InMemoryEventStore.generate(publisher=lambda e: None)
repository = Repository(event_store=event_store)
context = pmap({'repository': repository})
handler = TodoCommandHandler(context=context)

In [7]:
# Add some new todos, saving the resulting aggregates for
# their auto-generated ids
commands = pvector([
    TodoCommands.add(description='Todo 1', status='Queued'),
    TodoCommands.add(description='Todo 2', status='Queued'),
    TodoCommands.add(description='Todo 3', status='Queued'),
    TodoCommands.add(description='Todo 4', status='Queued'),
    TodoCommands.add(description='Todo 5', status='Queued'),
    TodoCommands.add(description='Todo 6', status='Queued'),
])
todos = list(map(handler.handle_command, commands))
for todo in todos:
    _data = thaw(todo.state)
    _data.update({'id': todo.id, 'version': todo.version})
    pprint(_data)

{'added_ts': datetime.datetime(2018, 6, 6, 5, 9, 50, 758019),
 'description': 'Todo 1',
 'id': '98fe6ce4-8f97-40ea-912e-34ef2421072b',
 'status': 'Queued',
 'version': 1}
{'added_ts': datetime.datetime(2018, 6, 6, 5, 9, 50, 758544),
 'description': 'Todo 2',
 'id': '3f0a784f-721e-45d3-86dd-2b81a08fdd7b',
 'status': 'Queued',
 'version': 1}
{'added_ts': datetime.datetime(2018, 6, 6, 5, 9, 50, 758987),
 'description': 'Todo 3',
 'id': '38cb75e4-53a3-43d9-88d1-c89bf672919b',
 'status': 'Queued',
 'version': 1}
{'added_ts': datetime.datetime(2018, 6, 6, 5, 9, 50, 759422),
 'description': 'Todo 4',
 'id': 'd3e135b2-f045-46cc-b4b1-dce73c4c876e',
 'status': 'Queued',
 'version': 1}
{'added_ts': datetime.datetime(2018, 6, 6, 5, 9, 50, 759870),
 'description': 'Todo 5',
 'id': '74d5eb32-a8d0-4c50-a59c-7f6b0713f237',
 'status': 'Queued',
 'version': 1}
{'added_ts': datetime.datetime(2018, 6, 6, 5, 9, 50, 760308),
 'description': 'Todo 6',
 'id': '90860531-af95-44ef-97b3-019a0e49e376',
 'status':

In [8]:
# Change the last 3 todos to a "Started" status, note the new status and changed_ts
change_commands = pvector([
    TodoCommands.change_status(todo.id, 'Started') for todo in todos[-3:]
])
changed_todos = list(map(handler.handle_command, change_commands))
for todo in changed_todos:
    _data = thaw(todo.state)
    _data.update({'id': todo.id, 'version': todo.version})
    pprint(_data)

{'added_ts': datetime.datetime(2018, 6, 6, 5, 9, 50, 759422),
 'changed_ts': datetime.datetime(2018, 6, 6, 5, 9, 50, 779359),
 'description': 'Todo 4',
 'id': 'd3e135b2-f045-46cc-b4b1-dce73c4c876e',
 'status': 'Started',
 'version': 2}
{'added_ts': datetime.datetime(2018, 6, 6, 5, 9, 50, 759870),
 'changed_ts': datetime.datetime(2018, 6, 6, 5, 9, 50, 779964),
 'description': 'Todo 5',
 'id': '74d5eb32-a8d0-4c50-a59c-7f6b0713f237',
 'status': 'Started',
 'version': 2}
{'added_ts': datetime.datetime(2018, 6, 6, 5, 9, 50, 760308),
 'changed_ts': datetime.datetime(2018, 6, 6, 5, 9, 50, 780555),
 'description': 'Todo 6',
 'id': '90860531-af95-44ef-97b3-019a0e49e376',
 'status': 'Started',
 'version': 2}


In [9]:
# Now iterate over the initial set of added todos and fetch
# the current state from the repository and pretty print it
for todo in todos:
    _todo = repository.get_aggregate(Todo, todo.id)
    _data = thaw(_todo.state)
    _data.update({'id': _todo.id, 'version': _todo.version})
    pprint(_data)

{'added_ts': datetime.datetime(2018, 6, 6, 5, 9, 50, 758019),
 'description': 'Todo 1',
 'id': '98fe6ce4-8f97-40ea-912e-34ef2421072b',
 'status': 'Queued',
 'version': 1}
{'added_ts': datetime.datetime(2018, 6, 6, 5, 9, 50, 758544),
 'description': 'Todo 2',
 'id': '3f0a784f-721e-45d3-86dd-2b81a08fdd7b',
 'status': 'Queued',
 'version': 1}
{'added_ts': datetime.datetime(2018, 6, 6, 5, 9, 50, 758987),
 'description': 'Todo 3',
 'id': '38cb75e4-53a3-43d9-88d1-c89bf672919b',
 'status': 'Queued',
 'version': 1}
{'added_ts': datetime.datetime(2018, 6, 6, 5, 9, 50, 759422),
 'changed_ts': datetime.datetime(2018, 6, 6, 5, 9, 50, 779359),
 'description': 'Todo 4',
 'id': 'd3e135b2-f045-46cc-b4b1-dce73c4c876e',
 'status': 'Started',
 'version': 2}
{'added_ts': datetime.datetime(2018, 6, 6, 5, 9, 50, 759870),
 'changed_ts': datetime.datetime(2018, 6, 6, 5, 9, 50, 779964),
 'description': 'Todo 5',
 'id': '74d5eb32-a8d0-4c50-a59c-7f6b0713f237',
 'status': 'Started',
 'version': 2}
{'added_ts': da

In [10]:
# Now let's remove one of the todos
removed_todo = handler.handle_command(TodoCommands.remove(id=todos[0].id))
_data = thaw(removed_todo.state)
_data.update({'id': removed_todo.id, 'version': removed_todo.version})
pprint(_data)

{'added_ts': datetime.datetime(2018, 6, 6, 5, 9, 50, 758019),
 'description': 'Todo 1',
 'id': '98fe6ce4-8f97-40ea-912e-34ef2421072b',
 'removed_ts': datetime.datetime(2018, 6, 6, 5, 9, 50, 816472),
 'status': 'Queued',
 'version': 2}


In [11]:
# We can technically skip the commands if we want to mainpulate
# things directly
todo = repository.get_aggregate(Todo, todos[-1].id)
todo = repository.save_aggregate(
    todo.change_status('Queued')\
        .change_status('Started')\
        .change_status('Finished')\
        .remove()
)
_data = thaw(todo.state)
_data.update({'id': todo.id, 'version': todo.version})
pprint(_data)
pprint([(e.type, thaw(e.data), e.timestamp) for e in todo.events])

{'added_ts': datetime.datetime(2018, 6, 6, 5, 9, 50, 760308),
 'changed_ts': datetime.datetime(2018, 6, 6, 5, 9, 50, 837242),
 'description': 'Todo 6',
 'id': '90860531-af95-44ef-97b3-019a0e49e376',
 'removed_ts': datetime.datetime(2018, 6, 6, 5, 9, 50, 837498),
 'status': 'Finished',
 'version': 6}
[('TodoAdded',
  {'description': 'Todo 6', 'status': 'Queued'},
  datetime.datetime(2018, 6, 6, 5, 9, 50, 760308)),
 ('TodoStatusChanged',
  {'id': '90860531-af95-44ef-97b3-019a0e49e376', 'status': 'Started'},
  datetime.datetime(2018, 6, 6, 5, 9, 50, 780555)),
 ('TodoStatusChanged',
  {'id': '90860531-af95-44ef-97b3-019a0e49e376', 'status': 'Queued'},
  datetime.datetime(2018, 6, 6, 5, 9, 50, 836275)),
 ('TodoStatusChanged',
  {'id': '90860531-af95-44ef-97b3-019a0e49e376', 'status': 'Started'},
  datetime.datetime(2018, 6, 6, 5, 9, 50, 836824)),
 ('TodoStatusChanged',
  {'id': '90860531-af95-44ef-97b3-019a0e49e376', 'status': 'Finished'},
  datetime.datetime(2018, 6, 6, 5, 9, 50, 837242)),